In [ ]:
# Authenticate GEE API
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import datetime
from dateutil.rrule import rrule, MONTHLY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geemap
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
code_title = 't1-buffer' # Will be part of a saved file names

# Import GEE assets
s2_col = ee.ImageCollection("COPERNICUS/S2_SR")
modis_col = ee.ImageCollection("MODIS/MOD09GA_006_NDVI")
forest_areas = ee.FeatureCollection('projects/id-ndvi-forest-project/assets/forest_areas')

fidn = 35 # Change based on fid of interest

fid = 'fid == ' + str(fidn) # Variable used for filtering forest_areas feature collection

area = forest_areas.filter(fid).geometry()
area_buffer = forest_areas.filter(fid).geometry().buffer(150)
aoi = area_buffer.difference(area) # Area of interest with original forest area cut out

gk = (forest_areas.filter(fid).aggregate_array('geokodas').getInfo())[0]

alldata = [] # All data will be added here

date_lst = [] # Dates will be added here

start_date = datetime.date(2017, 6, 1)
end_date = datetime.date(2022, 8, 1)

# Generates a list of dates
for d in rrule(MONTHLY, dtstart = start_date, until = end_date):
    date_lst.append(d.strftime('%Y-%m-%d'))

# Function for clipping to bounds of aoi
def imgclip(mask_clip):
    img_clip = mask_clip.clip(aoi)
    return img_clip

# Function for calculating NDVI
def imgndvi(mask_ndvi):
    img_ndvi = mask_ndvi.normalizedDifference(['B8', 'B4'])
    return img_ndvi

# Function for selecting s2 SCL vegetation band
def vegmask(mask_veg):
    img_vegmask = mask_veg.select('SCL').eq(4)
    return img_vegmask

# Function for selecting s2 SCL soil band
def soilmask(mask_so):
    img_soilmask = mask_so.select('SCL').eq(5)
    return img_soilmask

n = 2 # Number of dates to combine to a list, example n = 2, date_time = ['2019-01-01', '2019-02-01']

no = 1 # Month number counter

print('### calculation start ###')

for i in range(len(date_lst)-n+1):
    date_time = date_lst[i:i+n]
    
    date_stime = date_time[0]
    
    date_etime = date_time[1]
    
    s2_img = s2_col \
    .filterDate(date_stime, date_etime) \
    .select('B8', 'B4') \
    .filterBounds(aoi) \
    .map(imgclip) \
    .map(imgndvi)
    
    s2_scl = s2_col \
    .filterDate(date_stime, date_etime) \
    .select('SCL') \
    .filterBounds(aoi) \
    .map(imgclip)

    modis_img = modis_col \
    .filterDate(date_stime, date_etime) \
    .filterBounds(aoi)
    
    # Calculating mean values of aoi
    s2_veg_mean = s2_scl.map(vegmask).mean()

    s2_so_mean = s2_scl.map(soilmask).mean()
    
    modis_mean = modis_img.mean()
    
    s2_img_mean = s2_img.mean()
    
    # Calculating mean number of aoi
    veg_mean_num = s2_veg_mean.reduceRegion(reducer = ee.Reducer.mean(), geometry = aoi, scale = 20).getNumber('SCL').getInfo()
    
    so_mean_num = s2_so_mean.reduceRegion(reducer = ee.Reducer.mean(), geometry = aoi, scale = 20).getNumber('SCL').getInfo()
    
    modis_mean_num = modis_mean.reduceRegion(reducer = ee.Reducer.mean(), geometry = aoi, scale = 20).getNumber('NDVI').getInfo()
    
    s2_mean_num = s2_img_mean.reduceRegion(reducer = ee.Reducer.mean(), geometry = aoi, scale = 20).getNumber('nd').getInfo()
    
    data = {'no': no, 'date_time': date_stime[:-3], 'veg': veg_mean_num, 'so': so_mean_num, 'modis': modis_mean_num, 's2_ndvi': s2_mean_num, 'fid': fid, 'gk': gk}
    
    alldata.append(data)
    
    no += 1
    
    print(date_stime + ' ' + 'complete!')
    
print('### calculation end ###')

df_alldata = pd.DataFrame(alldata)
    
df_alldata.to_csv('./' + code_title + str(fidn) + '_df_alldata.csv')    
    
print('\ncsv table saved!')

x = df_alldata['date_time'].values.tolist()

s2_ndvi = df_alldata['s2_ndvi'].values.tolist()
modis = df_alldata['modis'].values.tolist()

veg = df_alldata['veg'].values.tolist()
soil = df_alldata['so'].values.tolist()

plt.subplot(2, 1, 1)
plt.xticks(np.arange(0, len(x), 6), rotation = 45)
plt.plot(x, s2_ndvi, color = 'lime')
plt.plot(x, veg, color = 'seagreen')
plt.legend(['s2_ndvi', 'modis'])

plt.subplot(2, 1, 2)
plt.xticks(np.arange(0, len(x), 6), rotation = 45)
plt.plot(x, veg, color = 'springgreen')
plt.plot(x, soil, color = 'brown')
plt.legend(['veg', 'soil'])

plt.tight_layout(pad=1.5)

plt.savefig('./' + code_title + str(fidn) + '_plot.png')

print('\nplots saved!')

arr = df_alldata[['s2_ndvi']].to_numpy()

result = seasonal_decompose(arr, period = 12, model = 'additive')

fig = result.plot()

fig.set_size_inches((8, 10))

fig.savefig('./' + code_title + str(fidn) + '_stats.png')

print('\nstats saved!')

Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)

Map.add_basemap('HYBRID')

Map.addLayer(aoi, {'color': '#DC143C'}, 'aoi', opacity = 0.6)

Map.centerObject(aoi, 17)

Map.to_html(filename = './' + code_title + fid +'_map.html', title = code_title + fid + '_map', width = '100%', height = '800px')

print('\nhtml map saved!')

print('\nfin :)')